In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [10]:
mnist.train.images.shape

(55000, 784)

In [11]:
784 ** .5

28.0

In [13]:
trainx = mnist.train.images

In [14]:
testx = mnist.test.images

In [15]:
trainy = mnist.train.labels

In [16]:
testy = mnist.test.labels

In [17]:
trainx.shape, testx.shape, trainy.shape, testy.shape

((55000, 784), (10000, 784), (55000, 10), (10000, 10))

In [18]:
trainx = trainx.reshape(-1, 1, 28, 28)
testx = testx.reshape(-1, 1, 28, 28)
trainx.shape, testx.shape

((55000, 1, 28, 28), (10000, 1, 28, 28))

In [33]:
class ResLayer(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size, padding, stride = 1, num_resid = 3):
        super(ResLayer, self).__init__()
        self.conv = nn.Conv2d(in_dim, out_dim, kernel_size, padding = padding, stride = stride)
        self.res = nn.Conv2d(out_dim, out_dim, kernel_size, padding = padding)
        self.num_resid = num_resid
        
    def forward(self, x):
        x = F.relu(self.conv(x))
        for _ in range(self.num_resid):
            x_conv = F.relu(self.res(x))
            x = F.relu(x + self.res(x_conv))
        
        return x

class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv1 = ResLayer(1, 32, 5, padding = 2)
        self.conv2 = ResLayer(32, 64, 5, padding = 2, stride = 2)
        self.conv3 = ResLayer(64, 128, 5, padding = 2, stride = 2)
        self.conv_out = nn.Conv2d(128, 10, 7, padding = 0)
        
        self.layers = nn.Sequential(self.conv1, self.conv2, self.conv3, self.conv_out, nn.Softmax(1))
    
    def forward(self, images):
        return self.layers(images).squeeze()

In [34]:
model = MNISTNet()
model(torch.FloatTensor(trainx[:5]))

tensor([[ 0.1019,  0.0974,  0.0999,  0.1040,  0.0997,  0.1012,  0.1036,
          0.1037,  0.0914,  0.0971],
        [ 0.1029,  0.0963,  0.0978,  0.1018,  0.0995,  0.1023,  0.1049,
          0.1040,  0.0903,  0.1003],
        [ 0.1008,  0.0962,  0.1002,  0.1022,  0.1016,  0.1009,  0.1031,
          0.1047,  0.0915,  0.0989],
        [ 0.1028,  0.0956,  0.0991,  0.1022,  0.1007,  0.1012,  0.1031,
          0.1050,  0.0925,  0.0979],
        [ 0.1036,  0.0966,  0.0982,  0.1013,  0.1021,  0.1010,  0.1021,
          0.1041,  0.0933,  0.0976]])

In [1]:
import numpy as np

In [ ]:
lrs = np.logspace(-5, 0)
losses = []
sample_indices = np.random.choice(range(10000), size = 128, replace = False)

test_data = torch.FloatTensor(testx[sample_indices])
data_labels = torch.LongTensor(trainy[sample_indices]).argmax(1)
test_data_labels = torch.LongTensor(testy[sample_indices]).argmax(1)

for i, lr in enumerate(lrs, start = 1):
    print(f"{i}: LR = {lr:.6f}")
    data = torch.FloatTensor(trainx[sample_indices]).requires_grad_(True)
    torch.manual_seed(1)
    model = MNISTNet()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    output = model(data)
    loss = F.cross_entropy(output, data_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    test_output = model(test_data)
    test_loss = F.cross_entropy(test_output, test_data_labels)
    losses.append(test_loss)

1: LR = 0.000010
2: LR = 0.000013
3: LR = 0.000016
4: LR = 0.000020
5: LR = 0.000026


ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "C:\Users\John\Anaconda3\envs\retro\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-51-b0abebd67f79>", line 15, in <module>
    output = model(data)
  File "C:\Users\John\Anaconda3\envs\retro\lib\site-packages\torch\nn\modules\module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-33-aed885b62452>", line 27, in forward
    return self.layers(images).squeeze()
  File "C:\Users\John\Anaconda3\envs\retro\lib\site-packages\torch\nn\modules\module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "C:\Users\John\Anaconda3\envs\retro\lib\site-packages\torch\nn\modules\container.py", line 91, in forward
    input = module(input)
  File "C:\Users\John\Anaconda3\envs\retro\lib\site-packages\torch\nn\modules\module.py